In [53]:
import numpy as np
from pydrake.geometry import StartMeshcat
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus,
    DoDifferentialInverseKinematics,
)
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, EventStatus, LeafSystem
from pydrake.visualization import MeshcatPoseSliders

from manipulation import running_as_notebook
from manipulation.meshcat_utils import WsgButton
from manipulation.scenarios import AddIiwaDifferentialIK, ExtractBodyPose
from manipulation.station import MakeHardwareStation, load_scenario

In [54]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7015


# Teleop Example (3D)

The physics and geometry engines running in the simulation above are actually running in 3D.  This example is almost identical, but we'll use the (default) 3D visualization and add more sliders for controlling the full `roll`, `pitch`, `yaw` angles and `x`, `y`, `z` positions of the end effector.

In [55]:
from constants import *
def make_stack():
    block_counter = -1
    output = ''
    for i in range(NUM_LAYERS):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(NUM_BLOCKS_PER_LAYER):
            x = (BLOCK_WIDTH * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (BLOCK_WIDTH * (j - 1))
            z = BLOCK_HEIGHT * (i + 0.5) + TABLE_HEIGHT
            block_counter += 1
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            print(f"block_{block_counter}")
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output


def make_optimal_stack():
    output = ''
    NUM_LAYERS = 54
    NUM_BLOCKS_PER_LAYER = 1
    block_counter = -1
    for i in range(NUM_LAYERS):
        layer_in_x_direction = i % 2 == 0
        for j in range(NUM_BLOCKS_PER_LAYER):
            x = (BLOCK_WIDTH * (j - 1) + BLOCK_WIDTH) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (BLOCK_WIDTH * (j - 1)) + BLOCK_WIDTH
            z = BLOCK_HEIGHT * (i + 0.5) + TABLE_HEIGHT
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            block_counter += 1
            print(f"block_{block_counter}")
            output += f"""
- add_model:
    name: block_{block_counter}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output
        

def construct_scenario():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
# plant_config:
#    time_step: 0.01

    scenario_data += make_optimal_stack()
    scenario = load_scenario(data = scenario_data)
    return scenario

In [56]:
def teleop_3d():
    meshcat.ResetRenderMode()
    builder = DiagramBuilder()
    scenario = construct_scenario()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml']))
    plant = station.GetSubsystemByName("plant")
    #station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    # TODO(russt): Replace with station.AddDiffIk(...)

    # Note: This is using "Cheat Ports". For it to work on hardware, we would
    # need to construct the initial pose from the HardwareStation outputs.


    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context()
    simulator_context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(simulator_context)
    for i in range(NUM_LAYERS*NUM_BLOCKS_PER_LAYER):
        some_body = (plant.GetBodyByName("jenga_block_link", plant.GetModelInstanceByName(f"block_{i}")))
        some_body.Lock(plant_context)
    block30 = plant.GetBodyByName("jenga_block_link", plant.GetModelInstanceByName(f"block_{30}"))
    block30.Unlock(plant_context) 
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            print(simulator.get_context().get_continuous_state_vector())    
        meshcat.DeleteButton("Stop Simulation")

    else:
        simulator.AdvanceTo(0.1)


teleop_3d()

block_0
block_1
block_2
block_3
block_4
block_5
block_6
block_7
block_8
block_9
block_10
block_11
block_12
block_13
block_14
block_15
block_16
block_17
block_18
block_19
block_20
block_21
block_22
block_23
block_24
block_25
block_26
block_27
block_28
block_29
block_30
block_31
block_32
block_33
block_34
block_35
block_36
block_37
block_38
block_39
block_40
block_41
block_42
block_43
block_44
block_45
block_46
block_47
block_48
block_49
block_50
block_51
block_52
block_53
Press Escape to stop the simulation
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e32ce10-e406-4026-abfb-00be78c478cb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>